In [21]:
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import scipy.spatial.distance as spdist

# Normalize the data.
from sklearn import preprocessing
from numpy.linalg import cholesky, det, lstsq
from scipy.optimize import minimize
from numpy.linalg import inv

def pass_arg(Xx, nsim, tr_size):
    
    # Compute the RMSE
    def root_mean_squared_error(y_true, y_pred):
        return np.sqrt(np.mean((y_pred-y_true)**2))

    print("tr_Size:",tr_size)
    # Making sure final porosity is less than initial
    def poros(poroi, porof):
        porofn = -porof*(porof<0)
        porofp = porof*(porof>=poroi) - poroi*(porof>=poroi)
        return porofp+porofn


    # Load labeled data
    data = np.loadtxt('../data/labeled_data.dat')
    x_labeled = data[:, :2].astype(np.float64) # -2 because we do not need porosity predictions
    y_labeled = data[:, -2:-1].astype(np.float64) # dimensionless bond length and porosity measurements

    # normalize dataset with MinMaxScaler
    scaler = preprocessing.MinMaxScaler(feature_range=(0.0, 1.0))
    x_labeled = scaler.fit_transform(x_labeled)
    # y_labeled = scaler.fit_transform(y_labeled)

    tr_size = int(tr_size)

    # train and test data
    trainX, trainY = x_labeled[:tr_size,:], y_labeled[:tr_size]
    testX, testY = x_labeled[tr_size:,:], y_labeled[tr_size:]


    # trainY = np.transpose(trainY)
    # testY = np.transpose(testY)

    data_phyloss = np.loadtxt('../data/unlabeled_data_BK_constw_v2_1525.dat')
    x_unlabeled = data_phyloss[:, :]

    # initial porosity
    initporo = x_unlabeled[:, -1]

    x_unlabeled1 = x_unlabeled[:1303, :2]
    x_unlabeled2 = x_unlabeled[-6:, :2]
    x_unlabeled = np.vstack((x_unlabeled1,x_unlabeled2))

    x_unlabeled = scaler.fit_transform(x_unlabeled)
    init_poro1 = initporo[:1303]
    init_poro2 = initporo[-6:]
    init_poro = np.hstack((init_poro1,init_poro2))
    
    
    def covSEard(hyp=None, x=None, z=None):
        ''' Squared Exponential covariance function with Automatic Relevance Detemination
         (ARD) distance measure. The covariance function is parameterized as:

         k(x^p,x^q) = sf2 * exp(-(x^p - x^q)' * inv(P) * (x^p - x^q)/2)

         where the P matrix is diagonal with ARD parameters ell_1^2,...,ell_D^2, where
         D is the dimension of the input space and sf2 is the signal variance.

         The hyperparameters are:

         hyp = [ log(ell_1)
                 log(ell_2)
                 ...
                 log(ell_D)
                 log(sqrt(sf2)) ]
        '''

        [n, D] = x.shape
        ell = 1/np.array(hyp[0:D])        # characteristic length scale
        
        
        sf2 = np.array(hyp[D])**2         # signal variance
        tmp = np.dot(np.diag(ell),x.T).T
        A = spdist.cdist(np.dot(np.diag(ell),x.T).T, np.dot(np.diag(ell),z.T).T, 'sqeuclidean') # cross covariances
        A = sf2*np.exp(-0.5*A)  

        return A


    def posterior_predictive(X_s, X_train, Y_train, l1=.1, l2=.1, sigma_f=.1, sigma_y=1e-5):
        '''  
        Computes the suffifient statistics of the GP posterior predictive distribution 
        from m training data X_train and Y_train and n new inputs X_s.

        Args:
            X_s: New input locations (n x d).
            X_train: Training locations (m x d).
            Y_train: Training targets (m x 1).
            l: Kernel length parameter.
            sigma_f: Kernel vertical variation parameter.
            sigma_y: Noise parameter.

        Returns:
            Posterior mean vector (n x d) and covariance matrix (n x n).
        '''


        K = covSEard(hyp=[l1,l2,sigma_f], x=X_train, z=X_train) + sigma_y**2 * np.eye(len(X_train))
        K_s = covSEard(hyp=[l1,l2,sigma_f], x=X_train, z=X_s)
        K_ss = covSEard(hyp=[l1,l2,sigma_f], x=X_s, z=X_s)  + 1e-8 * np.eye(len(X_s))
#         K_inv = inv(K)
        K_inv = np.linalg.pinv(K)
    
        # Equation (4)
        mu_s = K_s.T.dot(K_inv).dot(Y_train)

        # Equation (5)
        cov_s = K_ss - K_s.T.dot(K_inv).dot(K_s)
        
        return mu_s, cov_s


    def nll_fn(X_train, Y_train, x_unlabeled, init_poro, naive=False):
        '''
        Returns a function that computes the negative log marginal
        likelihood for training data X_train and Y_train and given 
        noise level.

        Args:
            X_train: training locations (m x d).
            Y_train: training targets (m x 1).
            noise: known noise level of Y_train.
            naive: if True use a naive implementation of Eq. (7), if 
                   False use a numerically more stable implementation. 

        Returns:
            Minimization objective.
        '''

        def nll_stable(theta):
            # Numerically more stable implementation of Eq. (7) as described
            # in http://www.gaussianprocess.org/gpml/chapters/RW2.pdf, Section
            # 2.2, Algorithm 2.1.
            K = covSEard(hyp=[theta[0],theta[1],theta[2]], x=X_train, z=X_train) + \
                theta[3]**2 * np.eye(len(X_train))
            
            
            K += 1e-6 * np.eye(*K.shape)
            L = cholesky(K)
        

            mu_un, _ = posterior_predictive(x_unlabeled, X_train, Y_train, l1=theta[0], l2=theta[1], sigma_f=theta[2], sigma_y=theta[3])
            phyloss_poro = np.mean(poros(init_poro, mu_un))

            log_loss = np.sum(np.log(np.diagonal(L))) + \
                   0.5 * Y_train.T.dot(lstsq(L.T, lstsq(L, Y_train)[0])[0]) + \
                   0.5 * len(X_train) * np.log(2*np.pi)
        
            # print(500000*phyloss_poro,log_loss, theta)
            return 500000*phyloss_poro + log_loss

        if naive:
            return nll_naive
        else:
            return nll_stable

    
    # Optimization
    res = minimize(nll_fn(trainX, trainY, x_unlabeled, init_poro), x0 = [.1, .1, .1, 1e-3], 
                   bounds=((1e-5, None), (1e-5, None), (1e-5, None),(1e-7, None)),
                    method='L-BFGS-B')
    
#     print(f'After parameter optimization: l1={res.x[0]:.5f} l2={res.x[1]:.5f} sigma_f={res.x[2]:.5f}')
#     print(np.exp(res.x[0]),np.exp(res.x[1]), np.exp(res.x[2]))
    mu_s, cov_s = posterior_predictive(testX, trainX, trainY, *res.x)
    
    RMSE = []
    for ii in range(int(nsim)):
        samples = np.random.multivariate_normal(mu_s.ravel(), cov_s, 1)
        RMSE.append(root_mean_squared_error(testY, samples))
        
        print("RMSE:", root_mean_squared_error(testY, samples))


    return samples, RMSE

In [22]:
Xx = np.random.uniform(size=(3, 2))
ss, rmse = pass_arg(Xx, 100, 30)

tr_Size: 30


C:\Users\berkc\Miniconda3\envs\R\lib\site-packages\ipykernel_launcher.py:160: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


RMSE: 0.01887178119891527
RMSE: 0.022422125472662455
RMSE: 0.01709485051420006
RMSE: 0.01894643997926025
RMSE: 0.024926161122790655
RMSE: 0.02088521855032978
RMSE: 0.02123508813025256
RMSE: 0.017469613535328905
RMSE: 0.02082993830179254
RMSE: 0.019368945066724246
RMSE: 0.021151389805499684
RMSE: 0.016658314991575485
RMSE: 0.021130084199330715
RMSE: 0.01908215000508387
RMSE: 0.01939684132909302
RMSE: 0.020203666011849638
RMSE: 0.01970623097466492
RMSE: 0.0190532637895618
RMSE: 0.018216483685318384
RMSE: 0.01974181719310583
RMSE: 0.02034516659833612
RMSE: 0.018584352201264587
RMSE: 0.02271928636080908
RMSE: 0.017779981884677682
RMSE: 0.024036591140256997
RMSE: 0.019111905320128877
RMSE: 0.02280856459857227
RMSE: 0.017998203325819247
RMSE: 0.018034947546228416
RMSE: 0.019466857744585567
RMSE: 0.021510992271884966
RMSE: 0.01890676622351696
RMSE: 0.020457298453895458
RMSE: 0.01941486616834205
RMSE: 0.020304294711812124
RMSE: 0.019659313038087448
RMSE: 0.017278456134758792
RMSE: 0.0187872673

In [23]:
np.mean(rmse)

0.02002908291162017